In [1]:
from estnltk import Text
import csv
from datetime import datetime
from tqdm import tqdm
import psycopg2
from connection_string_p12 import connection_string1, connection_string2
from collections import Counter
import pickle
import re

In [2]:
conn_egv = psycopg2.connect(connection_string1)
cur_egv = conn_egv.cursor()

In [3]:
cur_egv.execute("""SELECT diag_text_raw, diag_code from work.runfull201903041255_diagnosis 
                where diag_text_raw is not null;""")

In [4]:
diag_name_and_code = cur_egv.fetchall()

In [5]:
def find_diag_group(d):
    if d[0] == 'A' or d[0] == 'B':
        return 'A00-B99'
    elif d[0] == 'C' or (d[0] == 'D' and int(d[1]) < 5 ):
        return 'C00-D48'
    elif d[0] == 'D' and int(d[1]) >= 5:
        return 'D50-D89'
    elif d[0] == 'E':
        return 'E00-E90'
    elif d[0] == 'F':
        return 'F00-F99'
    elif d[0] == 'G':
        return 'G00-G99'
    elif d[0] == 'H' and int(d[1]) < 6:
        return 'H00-H59'
    elif d[0] == 'H' and int(d[1]) >= 6:
        return 'H60-H95'
    elif d[0] == 'I':
        return 'I00-I99'
    elif d[0] == 'J':
        return 'J00-J99'
    elif d[0] == 'K':
        return 'K00-K93'
    elif d[0] == 'L':
        return 'L00-L99'
    elif d[0] == 'M':
        return 'M00-M99'
    elif d[0] == 'N':
        return 'N00-N99'
    elif d[0] == 'O':
        return 'O00-O99'
    elif d[0] == 'P':
        return 'P00-P96'
    elif d[0] == 'Q':
        return 'Q00-Q99'
    elif d[0] == 'R':
        return 'R00-R99'
    elif d[0] == 'S' or d[0] == 'T':
        return 'S00-T98'
    elif d[0] == 'U':
        return 'U00-U99'
    elif d[0] == 'V' or d[0] == 'W' or d[0] == 'X' or d[0] == 'Y':
        return 'V01-Y98'
    elif d[0] == 'Z':
        return 'Z00-Z99'

In [6]:
diag_name_raw_groups = {}
for i in diag_name_and_code:
    try:
        d = find_diag_group(i[1])
        diag_name_raw_groups[i[0]] = d
    except:
        d = 'None'
        diag_name_raw_groups[i[0]] = d

In [7]:
len(diag_name_raw_groups)

102332

In [8]:
diag_name_raw = []
for i in diag_name_and_code:
    diag_name_raw.append(i[0])
diag_name_counts = Counter(diag_name_raw) 
diag_name_raw = set(diag_name_raw)

In [9]:
# erinevate diagnoosinimede arv
len(diag_name_raw)

102332

In [10]:
diag_name_counts.most_common()

[('Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta',
  11517),
 ('Hüpertooniatõbi e essentsiaalne arteriaalne hüpertensioon', 10235),
 ('Kodade virvendus ja laperdus', 9398),
 ('Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega',
  6291),
 ('Läbivaatus juhiloa saamiseks', 6079),
 ('Töötervishoiualane läbivaatus', 6012),
 ('Südamekahjustusega hüpertooniatõbi', 5303),
 ('Jälgimine muude kahtlustatud haiguste ja seisundite suhtes', 5007),
 ('Günekoloogiline läbivaatus (üldine; tava-)', 4740),
 ('Nimmevalu', 3970),
 ('Prostatahüperplaasia e eesnäärmesuurenemus', 3765),
 ('Kongestiivne südamepuudulikkus', 3630),
 ('Varasem müokardiinfarkt', 3598),
 ('Segatüüpi astma', 3517),
 ('Primaarne avatud- e lahtisenurga glaukoom', 3363),
 ('Karpaaltunneli sündroom', 3250),
 ('Liigsete kalorite põhjustatud rasvumus', 3142),
 ('Radikulopaatia', 3132),
 ('Insuliinisõltumatu suhkurtõbi', 3036),
 ('Segahüperlipideemia', 2912),
 ('Puhas hüperkolesteroleemia'

In [11]:
diag_names_correct_fixed = pickle.load(open("diag_names_correct_fixed.pickle", "rb")) 

In [47]:
diag_names_correct_fixed

{'Seksuaalkasvatus',
 'Hulgine ja kahepoolne pretserebraalarterite sulgus ja stenoos',
 'Kaaliumiliigsus',
 'Sõiduauto ukse või mõne muu osa tõttu lõksu sattunud (tegevus, mis on seotud muude eluavaldustega, nagu puhkamine, magamine, söömine jms)',
 'Torakoabdominaalaordi e rinna-kõhuaordi aneurüsmi rebend',
 'Seksuaalse arengu muud täpsustatud häired, heteroseksuaalsus',
 'Esmane krüoglobulineemia',
 'Liiklusvälises õnnetuses vigastatud täpsustamata pikapi- või veoautosõitja (muu täpsustamata tegevus)',
 'Difteriit[ili]ne larüngotrahheiit',
 'Ema hooldamine vililase (kahtlustatud) kahjustuse korral ema+punetistest',
 'Paratrahhoom',
 'Isiku kohtumine terviseteenistusega muudes täpsustatud olukordades',
 'Kaksteistsõrmiku-konstriktsioon',
 'Täpsustamata papuloskvamoosne haigusseisund',
 'Nakkuslik dermatiit',
 'Muud haavandilised koliidid e käärsoolepõletikud',
 'Jaotistes O00-O07 klassifitseeritud seisundite järgne+teisiti täpsustamata emboolia',
 'Täpsustamata difteeria',
 'Variveres

In [12]:
correct_names = {}
too_short_names = {}
too_long_names = []
nonexistant_names = {}
c = 0
for k, v in diag_name_counts.items():
    c += 1
    if c%100 == 0:
        print(c)
    if k in diag_names_correct_fixed:
        correct_names[k] = v
    else:
        for diag in diag_names_correct_fixed:
            if k in diag:
                too_short_names[k] = v
            elif diag in k:
                try:
                    diag = diag.replace('(', '\(')
                    diag = diag.replace(')', '\)')
                    too_long_names.append((diag, k, v, re.sub(diag, '', k)))
                except:
                    print(diag, k, v)
                #too_long_names[k] = v         
            else:
                nonexistant_names[k] = v
            

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [13]:
pickle.dump(too_long_names, open("too_long_diag_texts2.pickle", "wb"))

In [13]:
too_long_names = pickle.load(open("too_long_diag_texts.pickle", "rb"))

In [14]:
len(correct_names) + len(too_long_names) + len(nonexistant_names)

133346

In [15]:
len(correct_names)

4790

In [16]:
len(too_long_names)

31014

In [55]:
len(nonexistant_names)

97542

In [17]:
too_long_names

[('Fallopiuse juha', 'Fallopiuse juha PK', 2, ' PK'),
 ('Menopausi ja muud perimenopausaalsed haigusseisundid',
  'Menopausi ja muud perimenopausaalsed haigusseisundid, menopausijärgne verejooks',
  45,
  ', menopausijärgne verejooks'),
 ('Keisrilõikega üksiksünnitus',
  'Keisrilõikega üksiksünnitus, erakorraline keisrilõikesünnitus',
  53,
  ', erakorraline keisrilõikesünnitus'),
 ('Muud spondülopaatiad',
  'Muud spondülopaatiad, lülisambakanali kitsenemine e spinaalstenoos',
  114,
  ', lülisambakanali kitsenemine e spinaalstenoos'),
 ('Täpsustamata tekitajaga kopsupõletik',
  'Täpsustamata tekitajaga kopsupõletik, täpsustamata kopsupõletik',
  40,
  ', täpsustamata kopsupõletik'),
 ('Iseeneslik e spontaanne üksiksünnitus',
  'Iseeneslik e spontaanne üksiksünnitus, iseeneslik peaseis-sünnitus',
  173,
  ', iseeneslik peaseis-sünnitus'),
 ('Muud seropositiivsed reumatoidartriidid',
  'Muud seropositiivsed reumatoidartriidid  exacerbata',
  1,
  '  exacerbata'),
 ('Ema hooldamine vilil

In [48]:
too_long_names

[]

In [43]:
len(too_short_names)

99276

In [15]:
len(too_long_names)

31014

In [14]:
len(nonexistant_names)

97542

In [34]:
nonexistant_names

{('II funktsionaalne klass',): 72,
 ('II aste (organkahjustuse II staadium, kõrge riskigrupp) ',): 55,
 ('paroksüsmaalne',): 142,
 ('Adenocarcinoma prostatae',): 59,
 ('Carcinoma ventriculi recidiva rp T0N0M1 IV st',): 18,
 ('dekompensatsioon (kopsupais, bilateraalne pleuraefusioon, perifeersed tursed) --&gt; NYHA II aste',): 1,
 ('Gleason 3+3',): 8,
 ('Carcinoma recti cT4NxM1G2 IV st',): 54,
 ('paroksüsmaalne, ravitud (kardioversioon 15.07.2009)',): 2,
 ('lateraalne 10.07.2009',): 1,
 ('maksa desmoplastiline väike-ümarrakk sarkoom TxNxM1(lym, oss) st IV',): 2,
 ('0p. 2001',): 1,
 ('II  funktsionaalne klass ',): 9,
 ('inferioorne STEMI, 08.11.09',): 1,
 ('III st. NYHA j.',): 28,
 ('Grav. in h. 28+3; Glükoosi tolerantsuse häire',): 1,
 ('Grav in h. 37+1.Ebatuhud.',): 1,
 ('Muud ja Tta ravimid, medikamendid ja bioloogilised vahendid',): 1,
 ('Appendicitis acuta phlegmonosa',): 30,
 ('Kaudaalse regressiooni sündroom cauda equina kahjustusega.',): 1,
 ('Morbus Waldenströmi',): 8,
 ('Ca ves

In [15]:
len(diag_name_counts)

102332

In [18]:
extras = Counter()
extras2 = {}
for i in too_long_names:
    code_group = diag_name_raw_groups[i[1]]
    new_thing = (i[3], code_group)
    extras[new_thing] += i[2]

In [19]:
extras.most_common()

[(('\n', 'K00-K93'), 6636),
 (('\n', 'I00-I99'), 6165),
 (('\n', 'Z00-Z99'), 5932),
 (('\n', 'M00-M99'), 4022),
 (('\n', 'N00-N99'), 3636),
 (('\n', 'L00-L99'), 3477),
 (('\n', 'E00-E90'), 3245),
 (('\n', 'H00-H59'), 2328),
 (('\n', 'J00-J99'), 2139),
 (('\n', 'C00-D48'), 1831),
 (('\n', 'A00-B99'), 1333),
 (('\n', 'G00-G99'), 979),
 (('\n', 'R00-R99'), 919),
 (('\n', 'H60-H95'), 909),
 ((' (seedimatus)', 'K00-K93'), 876),
 ((', kodu, puhke- ja vaba aja tegevus', 'None'), 822),
 ((', jälgimine muude kahtlustatud haiguste ja seisundite suhtes', 'Z00-Z99'),
  811),
 ((' (N77.1*)', 'A00-B99'), 780),
 ((', jälgimine muude kahtlustatud haiguste ja seisundite suhtes\n',
   'Z00-Z99'),
  709),
 ((', emaka intramuraalne leiomüoom', 'C00-D48'), 668),
 ((', muu täpsustatud hüpotüreoos', 'E00-E90'), 653),
 ((', günekoloogiline läbivaatus (üldine; tava-)', 'Z00-Z99'), 639),
 (('\n', 'O00-O99'), 585),
 ((', imetava ema hooldus ja läbivaatus\n', 'Z00-Z99'), 574),
 (('\n', 'D50-D89'), 526),
 ((', kon

In [20]:
existing_diags = []
real_extras = Counter()
for i, c in extras.items():
    i_fix = i[0].strip('; ,\n')
    if len(i_fix) > 1:
        i_fix2 = i_fix[0].upper() + i_fix[1:]
        if i_fix2 in diag_names_correct_fixed:
            existing_diags.append((i_fix2, i[1]))
        else:
            real_extras[(i_fix2, i[1])] += c

In [21]:
len(existing_diags)

4249

In [22]:
real_extras.most_common()

[(('Kodu, puhke- ja vaba aja tegevus', 'None'), 1112),
 (('(seedimatus)', 'K00-K93'), 876),
 (('(N77.1*)', 'A00-B99'), 780),
 (('(emakasisese) kontratseptiivseadme järelevalve', 'Z00-Z99'), 660),
 (('Suulae- [kurgu-] ja [nina-]neelumandlite kroonilised haigused, krooniline mandlipõletik e tonsilliit',
   'J00-J99'),
  528),
 (('Ou', 'H00-H59'), 490),
 (('Tür[e]oidiit e kilpnäärmepõletik, autoimmuunne tür[e]oidiit', 'E00-E90'),
  489),
 (('PR', 'H00-H59'), 443),
 (('Oc. utr.', 'H00-H59'), 420),
 (('Od', 'H00-H59'), 390),
 (('Tänav ja [maan]tee, puhke- ja vaba aja tegevus', 'None'), 382),
 (('Os', 'H00-H59'), 366),
 (('Kodu', 'V01-Y98'), 321),
 (('(emakasisese) kontratseptiivseadme paigaldamine', 'Z00-Z99'), 285),
 (('Kodus puhke- või vabaaja tegevuse juures', 'None'), 258),
 (('Dermatofütoos e nahaseenhaigustu[mu]s, küünte nahaseenhaigustus e tiinea',
   'A00-B99'),
  250),
 (('II', 'I00-I99'), 247),
 (('E pneumoonia', 'J00-J99'), 235),
 (('Lampjalg (omandatud)pes planus (acquisitus)', 

In [23]:
pickle.dump(real_extras, open("real_extras2.pickle", "wb"))

In [64]:
one_word = Counter()
anonym = Counter()
date = Counter()
brackets = Counter()
abbreviations = Counter()
numbers = Counter()
string_freqs = Counter()
o = 0
a = 0
d = 0
b = 0
other = Counter()
for diag, c in real_extras.items():
    i = diag[0]
    group = diag[1]
    if len(i.split()) == 1:
        one_word[i] = c
        o = 1
    if 'ANONYM' in i:
        anonym[i] = c
        a = 1
    if re.search('[0-9][0-9]\.[0-9][0-9]\.[0-9]{4}', i):
        date[i] = c
        d = 1
    if i.startswith('('):
        brackets[i] = c  
        b = 1
    if o == 0 and a == 0 and d == 0 and b == 0:
        other[(i, group)] = c
    o = 0
    a = 0
    d = 0
    b = 0
    strings = i.split()
    for s in strings:
        string_freqs[s] += 1

In [26]:
one_word.most_common()

[('(seedimatus)', 876),
 ('(N77.1*)', 780),
 ('Ou', 662),
 ('Od', 520),
 ('Os', 475),
 ('PR', 443),
 ('Kahtlus', 346),
 ('Kodu', 284),
 ('Vasakul', 218),
 ('Paremal', 207),
 ('Dex', 166),
 ('Neerukivi', 156),
 ('F07.2', 154),
 ('Sin', 149),
 ('Paroksüsmaalne', 123),
 ('Kusepõiesong', 121),
 ('Pk', 110),
 ('OU', 99),
 ('OD', 95),
 ('(permanentne)', 91),
 ('Fikseerunud', 89),
 ('Oc.utr.', 85),
 ('Ägenemine', 80),
 ('OS', 74),
 ('(L40.5ü)', 71),
 ('Permanens', 70),
 ('Kliiniliselt', 67),
 ('Nimmepiirkond', 60),
 ('Aordi(klapi)stenoos', 57),
 ('Kaelapiirkond', 57),
 ('(paroksüsmaalne)', 56),
 ('(C00-D48)', 55),
 ('Raskekujuline', 54),
 ('II', 53),
 ('Parempoolne', 52),
 ('Sin.', 49),
 ('Oc.dex.', 49),
 ('(ägenemine)', 48),
 ('Kinnine', 47),
 ('Umbsool', 46),
 ('Statiinravil', 46),
 ('Sõltuvus', 44),
 ('(C00-D48Ś)', 44),
 ('Opereeritud', 44),
 ('Kergekujuline', 42),
 ('Oc.sin', 42),
 ('(-)', 41),
 ('Osteopeenia', 41),
 ('(G99.2*)', 39),
 ('Oc.sin.', 39),
 ('III', 38),
 ('(bronhiolektaasia)'

In [27]:
anonym.most_common()

[('. <ANONYM id="0" type="per" morph="_S_ sg n"/>', 61),
 ('. <ANONYM id="0" type="per" morph="_S_ sg n"/>, puhke- ja vaba aja tegevus',
  47),
 ('. <ANONYM id="0" type="per" morph="_S_ sg n"/> ja [maan]tee, puhke- ja vaba aja tegevus',
  36),
 ('<ANONYM id="1" type="per" morph="_H_ sg n"/> os', 29),
 ('<ANONYM id="1" type="per" morph="_H_ sg n"/> od', 16),
 ('<ANONYM id="0" type="per" morph="_S_ sg n"/> ja jalg', 15),
 ('. <ANONYM id="0" type="per" morph="_S_ sg n"/> ja [maan]tee', 15),
 ('<ANONYM id="1" type="per" morph="_H_ sg n"/>', 13),
 ('<ANONYM id="0" type="per" morph="_Y_ ?"/> A', 12),
 ('(<ANONYM id="1" type="per" morph="_Y_ ?"/>)', 8),
 ('(Emakasisese) kontratseptiivseadme järelevalve - <ANONYM id="1" type="per" morph="_H_ sg g"/>',
  8),
 ('Lampjalg (omandatud)Pes planus (acquisitus), <ANONYM id="0" type="per" morph="_S_ sg n"/> ja jalg',
  7),
 ('. <ANONYM id="0" type="per" morph="_S_ sg n"/> ja jalg', 7),
 ('<ANONYM id="1" type="per" morph="_Y_ ?"/> B, endine suitsetaja',

In [28]:
re.findall('id="([0-9]+)" ', 'I st <ANONYM id="1" type="per" morph="_S_ sg g"/> järgi,B st Binet järgi')

['1']

In [29]:
anonym_ids = Counter()
for i, c in anonym.items():
    ids = re.findall('id="([0-9]+)" ', i)
    for j in ids:
        anonym_ids[j] += c

In [30]:
anonym_ids.most_common()

[('1', 634), ('0', 515), ('2', 64), ('3', 47), ('4', 6), ('5', 2), ('7', 1)]

In [31]:
date.most_common()

[('(VVIR 70x 09.05.2011 )', 18),
 ('04.06.2012', 11),
 ('21.08.2007-vasaku puusaliigese endoproteesimine', 11),
 ('VVI ,fr.65 x min. 24.04.2014', 11),
 ('E pk -26.08.2013- adenocarcinoma tubulare coli sigmoideipT3N1cM0G2 st.IIIb-op. 04.09.20',
  10),
 ('(seinaläbine, 30.11.2010)', 10),
 ('01.05.2009 (esmane, kadaveerne)', 9),
 ('(TAP sin 18.09.2013)', 9),
 ('( 12.04.2010 )', 9),
 ('(07.05.06 infer.; 09.09.2010 STEMI anter.)', 9),
 ('( 01.12.2004; 21.04.2014 ; 26.05.2014 )', 9),
 ('03.04.2012 (korduv, kadaveerne)', 8),
 ('(TAP dex 06.10.2010)', 8),
 ('26.03.1996 (esmane, kadaveerve)', 6),
 ('(1999; vahetus-  DDDR 22.07.2013', 6),
 ('( par. puusaliiges  28.03.2012 )', 6),
 ('(2005; 19.07.2013)', 5),
 ('23.04.2002 (esmane, kadaveerne)', 5),
 ('A pk op.26.09.2011', 5),
 ('Peaajuinfarkt 08.04.2014', 5),
 ('( Bariaatriline op. 10.08.2012)', 5),
 ('( op. vasakul  14.03.2011 )', 5),
 ('30.12.2003 (esmane, kadaveerne)', 5),
 ('( 11.03.2014 )', 5),
 ('Op-tud 14.11.2012,   01.07. 2013', 5),
 ('. 

In [32]:
brackets.most_common()

[('(seedimatus)', 876),
 ('(N77.1*)', 780),
 ('(emakasisese) kontratseptiivseadme järelevalve', 660),
 ('(emakasisese) kontratseptiivseadme paigaldamine', 285),
 ('(E10-E14+ ühise neljanda tärgig', 183),
 ('(2)Südamekahjustusega hüpertooniatõbi ilma (kongestiivse) südamepuudulikkuseta',
  104),
 ('(permanentne)', 91),
 ('(krooniline) verekaotusjärgne teisene rauavaegusaneemia', 88),
 ('(E10-E14+ ühise neljanda tärgiga .', 74),
 ('(L40.5ü)', 71),
 ('(2)Südamekahjustusega hüpertooniatõbi (kongestiivse) südamepuudulikkusega',
  71),
 ('(paroksüsmaalne)', 56),
 ('(C00-D48)', 55),
 ('(ägenemine)', 48),
 ('(C00-D48Ś)', 44),
 ('(Emakasisese) kontratseptiivseadme järelevalve\nESS Mirena', 43),
 ('(-)', 41),
 ('(G99.2*)', 39),
 ('(perifeerne; krooniline) veenipuudulikkus', 38),
 ('(bronhiolektaasia)', 37),
 ('(dex)', 36),
 ('(persisteeruv)', 35),
 ('(E10-E14+ ühise neljanda', 32),
 ('(emakasisese) kontratseptiivseadme olemasolu', 31),
 ('(E10-E14  ühise neljanda tärgig', 24),
 ('(sin)', 23),
 (

In [33]:
string_freqs.most_common()

[('ja', 1143),
 ('(', 860),
 ('-', 832),
 ('type="per"', 768),
 ('<ANONYM', 732),
 ('.', 645),
 ('sg', 595),
 ('in', 520),
 ('e', 495),
 ('II', 486),
 ('morph="_H_', 448),
 ('id="1"', 444),
 ('n"/>', 435),
 ('hüpertooniatõbi', 392),
 ('(kongestiivse)', 391),
 ('Südamekahjustusega', 382),
 (')', 355),
 ('I', 354),
 ('dex', 319),
 ('III', 300),
 ('sin', 300),
 ('või', 298),
 ('ilma', 246),
 ('dex.', 242),
 (',', 237),
 ('id="0"', 232),
 ('sin.', 229),
 ('hebd', 214),
 ('paremal', 201),
 ('morph="_Y_', 196),
 ('2', 190),
 ('vasakul', 184),
 ('täpsustamata', 181),
 ('maturus', 179),
 ('Parema', 174),
 ('post', 167),
 ('NYHA', 166),
 ('et', 160),
 ('täpsustatud', 158),
 ('Grav', 158),
 ('hebd.', 157),
 ('Od', 154),
 ('muu', 154),
 ('st.', 151),
 ('/', 149),
 ('parem', 147),
 ('tegevus', 146),
 ('muud', 142),
 ('Os', 141),
 ('südamepuudulikkuseta', 141),
 ('os', 140),
 ('op.', 139),
 ('aste', 137),
 ('(Emakasisese)', 135),
 ('Vasaku', 132),
 ('HAS-BLED', 131),
 ('IV', 126),
 ('kasvaja', 126)

In [65]:
other.most_common()

[(('Kodu, puhke- ja vaba aja tegevus', 'None'), 994),
 (('Suulae- [kurgu-] ja [nina-]neelumandlite kroonilised haigused, krooniline mandlipõletik e tonsilliit',
   'J00-J99'),
  528),
 (('Tür[e]oidiit e kilpnäärmepõletik, autoimmuunne tür[e]oidiit', 'E00-E90'),
  489),
 (('Oc. utr.', 'H00-H59'), 401),
 (('Tänav ja [maan]tee, puhke- ja vaba aja tegevus', 'None'), 379),
 (('Dermatofütoos e nahaseenhaigustu[mu]s, küünte nahaseenhaigustus e tiinea',
   'A00-B99'),
  250),
 (('E pneumoonia', 'J00-J99'), 235),
 (('Lampjalg (omandatud)pes planus (acquisitus)', 'M00-M99'), 220),
 (('Tür[e]otoksikoos [hüpertüreoos], tür[e]otoksikoos difuusse struumaga',
   'E00-E90'),
  219),
 (('Kodus puhke- või vabaaja tegevuse juures', 'None'), 211),
 (('Muu spondüloos', 'M00-M99'), 186),
 (('Oc utr', 'H00-H59'), 183),
 (('Kodarluu ala- [distaalse] otsa murd, kinnine', 'S00-T98'), 182),
 (('Krooniline c-viirushepatiit', 'A00-B99'), 175),
 (('Viirustüükad [soolatüükad] e harilikud tüükad', 'A00-B99'), 127),
 

In [66]:
with open("other_diag_text_extras_with_codegroups.csv", "w") as fout:
    writer = csv.writer(fout)
    for row in other.most_common():
        writer.writerow([row[0][0], row[0][1], row[1]])